In [ ]:
"validity": {
        "AssessContext": {
            "valid_function_args": 0.6590730557737627,
            "valid_indicator": 0.783974862529458,
            "valid_json": 0.6590730557737627,
            "not_none_output": 0.6025137470542027
        },
        "Wikipedia": {
            "valid_function_args": 0.9732914375490966,
            "valid_indicator": 1.0,
            "valid_json": 0.9732914375490966,
            "not_none_output": 0.783974862529458
        }
    },

In [19]:
import plotly.graph_objects as go
# Adjusted data for the Sankey diagram to reflect the updated flow
labels = ["Wikipedia", "Summarize", "No search results","AssessContext", "Invalid function arguments",  "Relevant context", "Irrelevant context"]

# Updating sources and targets to reflect the new flow
source = [0, 0, 0, 1, 3, 3,  3]  # Wikipedia to outputs and AssessContext
target = [1, 4, 2, 3, 4, 5,  6]  # Outputs and Invalid/No Result

# Values - Assuming that the sum of outputs from Wikipedia is 1
value = [
    78.4,  # Wikipedia to AssessContext ('not_none_output')
    2.7,  
    18.9,  
    78.4,  # Wikipedia to Invalid/No Result (remainder for 'valid_json')
    12.4,  # AssessContext to 'valid_function_args'
    60.3,  # AssessContext to 'valid_indicator'
    5.7,  # AssessContext to 'valid_json'
]

# Creating the updated Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
      pad=15,
      thickness=20,
      line=dict(color="black", width=0.3),
      label=labels,
    ),
    link=dict(
      source=source,
      target=target,
      value=value
    ))])

fig.update_layout(title_text="Flow of the CoT agent", font_size=10)
fig.show()



In [106]:
import plotly.graph_objects as go

# Node data
mynode = dict(
    pad=15,
    thickness=20,
    line=dict(color="black", width=0.5),
    label=["Wikipedia: 100%", "Summarize: 78.4%", "No search results: 18.9%", "Filter: 78.4%", 
           "Invalid function arguments: 15.1%", "Relevant context: 60.3%", "Irrelevant context: 5.7%", "Correct answer: X%, Incorrect answer: Y%"],
    x=[0.1, 0.3, 0.3, 0.5, 0.7, 0.7, 0.7, 0.9,0.9],  # Adjust x positions
    y=[0.1, 0.1, 0.82, 0.1, 0.8, 0.1, 0.5, 0.1, 0.7],  # Adjust y positions
    color=[
"lightblue",#"rgba(127, 127, 127, 0.8)", 
"rgba(255, 127, 14, 0.5)",#"rgba(23, 190, 207, 0.8)" ,
"rgba(148, 103, 189, 0.8)" ,
"lightyellow" ,
"rgba(214, 39, 40, 0.8)",
"rgba(44, 160, 44, 0.8)",
"rgba(140, 86, 75, 0.8)" ]  # Adjust colors
)

# Link data
mylink = dict(
    source=[0, 0, 0, 1, 3, 3, 3],  # Source nodes
    target=[1, 4, 2, 3, 4, 5, 6],  # Target nodes
    value=[78.4, 2.7, 18.9, 78.4, 12.4, 60.3, 5.7],  # Values
)


# Creating the Sankey diagram with custom node positions and colors
fig = go.Figure(data=[go.Sankey(
    arrangement='snap',
    node=mynode,
    link=mylink
)])

fig.update_layout(title_text="Flow of the CoT agent", font_size=14)
fig.show()

In [65]:
import json
import pandas as pd
def load_file(file_path):
    # Attempt to load the file again, treating each line as a dictionary
    with open(file_path, 'r') as file:
        file_content_dicts = [json.loads(line) for line in file]


    # Convert the list of dictionaries to a pandas dataframe
    df = pd.DataFrame(file_content_dicts)
    return df


#WikiCoT = r"/Users/jensperregaardthorsen/git/rosetta/scratch/rosetta-results/USMLE/mixtral8x7b-instructv1/WikipediaCoTAgent/Prompt 11/Summarize True/Two-step False/Sumfilter True/14-01-24/5c0a66318f708e5c3cf877d963510dfe3c4cf35aee4b0f6b144dc7ae27ca62cf/dataset.jsonl"
WikiCoT = r"/Users/jensperregaardthorsen/git/rosetta/scratch/rosetta-results/USMLE/mixtral8x7b-instructv1/WikipediaCoTAgent/Prompt 11/Summarize True/Two-step False/Sumfilter True/13-01-24/5c0a66318f708e5c3cf877d963510dfe3c4cf35aee4b0f6b144dc7ae27ca62cf/dataset.jsonl"

In [71]:
df["validity"][0]["Wikipedia"]

{'not_none_output': True,
 'valid_function_args': True,
 'valid_indicator': True,
 'valid_json': True}

In [133]:
df = load_file(WikiCoT)

noSearch = 0
noSearchCorrect = 0
noSearchIncorrect = 0

SearchInvalidArgs = 0
SearchInvalidArgsCorrect = 0
SearchInvalidArgsIncorrect = 0

RelevantContext = 0
RelevantContextCorrect = 0
RelevantContextIncorrect = 0

IrrelevantContext = 0
IrrelevantContextCorrect = 0
IrrelevantContextIncorrect = 0

FilterInvalidArgs = 0
FilterInvalidArgsCorrect = 0
FilterInvalidArgsIncorrect = 0

# generate column for rows of interest
nosearch = []
searchinvalidargs = []
relevantcontext = []
irrelevantcontext = []
filterinvalidargs = []


for index, row in df.iterrows():
    #NO SEARCH
    if row["validity"]["Wikipedia"]["not_none_output"] == 0 and row["validity"]["Wikipedia"]["valid_function_args"] ==1:
        noSearch += 1
        nosearch.append(True)
        if row["exactmatch"] == True:
            noSearchCorrect += 1
        else:
            noSearchIncorrect += 1
    else:
        nosearch.append(False)

    # SEARCH INVALID ARGS
    if row["validity"]["Wikipedia"]["valid_function_args"] == 0:
        SearchInvalidArgs += 1
        searchinvalidargs.append(True)
        if row["exactmatch"] == True:
            SearchInvalidArgsCorrect += 1
            print(row["validity"])
        else:
            SearchInvalidArgsIncorrect += 1
    else:
        searchinvalidargs.append(False)

    # RELEVANT CONTEXT
    if row["validity"]["AssessContext"]["not_none_output"] == 1:
        RelevantContext += 1
        relevantcontext.append(True)
        if row["exactmatch"] == True:
            RelevantContextCorrect += 1
        else:
            RelevantContextIncorrect += 1
    else:
        relevantcontext.append(False)

    # IRRELEVANT CONTEXT
    if row["validity"]["AssessContext"]["not_none_output"] == 0 and row["validity"]["AssessContext"]["valid_function_args"] == 1:
        IrrelevantContext += 1
        irrelevantcontext.append(True)
        if row["exactmatch"] == True:
            IrrelevantContextCorrect += 1
        else:
            IrrelevantContextIncorrect += 1
    else:
        irrelevantcontext.append(False)

    if row["validity"]["AssessContext"]["valid_function_args"] == 0 and row["validity"]["Wikipedia"]["not_none_output"] == 1:
        FilterInvalidArgs += 1
        filterinvalidargs.append(True)
        if row["exactmatch"] == True:
            FilterInvalidArgsCorrect += 1
        else:
            FilterInvalidArgsIncorrect += 1
    else:
        filterinvalidargs.append(False)
    

    
    
print("NoSearch: ", noSearch/len(df))
print("NoSearchCorrect: ", noSearchCorrect/len(df))
print("NoSearchIncorrect: ", noSearchIncorrect/len(df))
print("")
print("SearchInvalidArgs: ", SearchInvalidArgs/len(df))
print("SearchInvalidArgsCorrect: ", SearchInvalidArgsCorrect/len(df))
print("SearchInvalidArgsIncorrect: ", SearchInvalidArgsIncorrect/len(df))
print("")
print("RelevantContext: ", RelevantContext/len(df))
print("RelevantContextCorrect: ", RelevantContextCorrect/len(df))
print("RelevantContextIncorrect: ", RelevantContextIncorrect/len(df))
print("")
print("IrrelevantContext: ", IrrelevantContext/len(df))
print("IrrelevantContextCorrect: ", IrrelevantContextCorrect/len(df))
print("IrrelevantContextIncorrect: ", IrrelevantContextIncorrect/len(df))
print("")
print("FilterInvalidArgs: ", FilterInvalidArgs/len(df))
print("FilterInvalidArgsCorrect: ", FilterInvalidArgsCorrect/len(df))
print("FilterInvalidArgsIncorrect: ", FilterInvalidArgsIncorrect/len(df))


{'AssessContext': {'not_none_output': False, 'valid_function_args': False, 'valid_indicator': False, 'valid_json': False}, 'Wikipedia': {'not_none_output': False, 'valid_function_args': False, 'valid_indicator': True, 'valid_json': False}}
{'AssessContext': {'not_none_output': False, 'valid_function_args': False, 'valid_indicator': False, 'valid_json': False}, 'Wikipedia': {'not_none_output': False, 'valid_function_args': False, 'valid_indicator': True, 'valid_json': False}}
{'AssessContext': {'not_none_output': False, 'valid_function_args': False, 'valid_indicator': False, 'valid_json': False}, 'Wikipedia': {'not_none_output': False, 'valid_function_args': False, 'valid_indicator': True, 'valid_json': False}}
{'AssessContext': {'not_none_output': False, 'valid_function_args': False, 'valid_indicator': False, 'valid_json': False}, 'Wikipedia': {'not_none_output': False, 'valid_function_args': False, 'valid_indicator': True, 'valid_json': False}}
{'AssessContext': {'not_none_output': Fa

In [119]:
# calculate the percentage of correct answers for each step
print("NoSearchCorrect: ", noSearchCorrect/noSearch)
print("SearchInvalidArgsCorrect: ", SearchInvalidArgsCorrect/SearchInvalidArgs)
print("RelevantContextCorrect: ", RelevantContextCorrect/RelevantContext)
print("IrrelevantContextCorrect: ", IrrelevantContextCorrect/IrrelevantContext)
print("FilterInvalidArgsCorrect: ", FilterInvalidArgsCorrect/FilterInvalidArgs)

#Total average
print("Total average: ", (noSearchCorrect+SearchInvalidArgsCorrect+RelevantContextCorrect+IrrelevantContextCorrect+FilterInvalidArgsCorrect)/len(df))


NoSearchCorrect:  0.5975103734439834
SearchInvalidArgsCorrect:  0.7647058823529411
RelevantContextCorrect:  0.681877444589309
IrrelevantContextCorrect:  0.5972222222222222
FilterInvalidArgsCorrect:  0.6352201257861635
Total average:  0.6575019638648861


In [128]:
# load baseline and WikiCoT and filter on relevant context
def merge_baseline_and_agent(file_baseline, file_agent):
    # Load the files
    df_baseline = load_file(file_baseline)
    df_agent = load_file(file_agent)

    # Merge the two dataframes on the id column
    df_merged = pd.merge(df_baseline, df_agent, on='id', suffixes=('_baseline', '_agent'))
    return df_merged

    
CoT = r"/Users/jensperregaardthorsen/git/rosetta/scratch/rosetta-results/USMLE/mixtral8x7b-instructv1/FewShotCoTAgent/Prompt 11/Summarize True/Two-step False/06-01-24/5ffc6b947ca08dfff3c2326dc1f17c82455da4429c719171799ac9f18fcc8283/dataset.jsonl"
WikiCoT = r"/Users/jensperregaardthorsen/git/rosetta/scratch/rosetta-results/USMLE/mixtral8x7b-instructv1/WikipediaCoTAgent/Prompt 11/Summarize True/Two-step False/Sumfilter True/13-01-24/5c0a66318f708e5c3cf877d963510dfe3c4cf35aee4b0f6b144dc7ae27ca62cf/dataset.jsonl"

In [134]:
df = merge_baseline_and_agent(CoT, WikiCoT)

# add relevantcontext list
df["relevantcontext"] = relevantcontext
df["irrelevantcontext"] = irrelevantcontext
df["filterinvalidargs"] = filterinvalidargs
df["searchinvalidargs"] = searchinvalidargs
df["nosearch"] = nosearch

# filter on relevant context


In [139]:
for prop in ["nosearch", "searchinvalidargs", "filterinvalidargs", "irrelevantcontext", "relevantcontext"]:
    df_ = df[df[prop] == True]
    print(prop)
    print(df_.shape)
    print("CoT Exactmatch", df_["exactmatch_baseline"].mean())
    print("WikiCoT Exacthmatch", df_["exactmatch_agent"].mean())
    print("")

print("average")
print("CoT Exactmatch", df["exactmatch_baseline"].mean())
print("WikiCoT Exacthmatch", df["exactmatch_agent"].mean())

nosearch
(241, 26)
CoT Exactmatch 0.5892116182572614
WikiCoT Exacthmatch 0.5975103734439834

searchinvalidargs
(34, 26)
CoT Exactmatch 0.7941176470588235
WikiCoT Exacthmatch 0.7647058823529411

filterinvalidargs
(159, 26)
CoT Exactmatch 0.6037735849056604
WikiCoT Exacthmatch 0.6352201257861635

irrelevantcontext
(72, 26)
CoT Exactmatch 0.5833333333333334
WikiCoT Exacthmatch 0.5972222222222222

relevantcontext
(767, 26)
CoT Exactmatch 0.6740547588005215
WikiCoT Exacthmatch 0.681877444589309

average
CoT Exactmatch 0.6472898664571878
WikiCoT Exacthmatch 0.6575019638648861


In [132]:
# calculate the average exactmatch


CoT Exactmatch 0.6740547588005215
WikiCoT Exacthmatch 0.681877444589309
